#import, 설치, 구글 드라이브 마운트

In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
import os

import torch
import torch.nn as nn
from tqdm import trange
from transformers import XLMRobertaModel, AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset
from transformers import get_linear_schedule_with_warmup
from transformers import AdamW
from datasets import load_metric
from sklearn.metrics import f1_score
import pandas as pd
import copy

import json
import pandas as pd

from collections import Counter

import re

# functions

json 및 jsonl 파일 read, write 함수

In [ ]:
def jsonload(fname, encoding="utf-8"):
    with open(fname, encoding=encoding) as f:
        j = json.load(f)
    return j

# json 개체를 파일이름으로 깔끔하게 저장
def jsondump(j, fname):
    with open(fname, "w", encoding="UTF8") as f:
        json.dump(j, f, ensure_ascii=False)

# jsonl 파일 읽어서 list에 저장
def jsonlload(fname, encoding="utf-8"):
    json_list = []
    with open(fname, encoding=encoding) as f:
        for line in f.readlines():
            json_list.append(json.loads(line))
    return json_list

앙상블 round 함수

        round 함수 : model list에서 n번째 모델을 하나씩 빼면서 새로운 리스트 만든다
        
        models = [A, B, C, D]

        models_1 = [B, C, D]
        models_2 = [A, C, D]
        models_3 = [A, B, D]
        models_4 = [A, B, C]

In [ ]:
def round(model_list):
    round_model_list = [None] * len(model_list)

    for idx in range(len(round_model_list)):
        round_model_list[idx] = model_list[:idx] + model_list[idx+1:]
    return round_model_list

excute_ensemble, save_jsonl

In [ ]:
def excute_ensemble(automatic_ae):
    dic_ae = {
        'id' : [],
        'sentence_form' : [],
        'annotation' : []
    }

    models = automatic_ae 


    for i in range(len(models[0])):
        tmp_divide = []

        for j in models:
            tmp_divide.append( str(j['annotation'][i]) )

        answer = Counter(tmp_divide).most_common(n=1)[0][0]

        check =  str(tmp_divide).replace("[", "").replace("]", "").replace(",", "").replace("\'", "").replace("\"", "").replace(" ", "")

        if '[]' in answer and check is not "":
            while '[]' in tmp_divide:
                tmp_divide.remove('[]')
            dic_ae['annotation'].append( tmp_divide[0] )
        else:
            dic_ae['annotation'].append( answer )
        dic_ae['id'].append( j['id'][i] )
        dic_ae['sentence_form'].append( j['sentence_form'][i] )

    df_ae = pd.DataFrame( dic_ae )

    return df_ae

In [ ]:
def save_jsonl(df_ae, name, path):

    with open("{1}/{0}.jsonl" .format(name, path), 'w') as file:
        for i in range( len(df_ae) ):
            annos = df_ae['annotation'][i]
            str_annos = str(annos)
            tmp = str_annos.replace("None", "null").replace("\'", "\"")

            file.write(  '{'+'\"id\": \"nikluge-sa-2022-{3}-{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                        .format( str(i+1).zfill(5)  ,   df_ae['sentence_form'][i], tmp, name ) +'}' )
            file.write("\n")

#{"id": "nikluge-sa-2022-dev-00001", "sentence_form": "깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면 금방 우러난다.", "annotation": [["본품#편의성", ["부직포 포장", 5, 11], "positive"]]}
# 0은 숫자, 1은 sentence, 2는 annotation, 3은 file_name(파일이름)

# Load Files

In [ ]:
### 모델 이름과 주소를 입력하세요

model_A = pd.DataFrame(jsonlload("/content/model_A.jsonl"))
model_B = pd.DataFrame(jsonlload("/content/model_B.jsonl"))
model_C = pd.DataFrame(jsonlload("/content/model_C.jsonl"))

# Start Ensemble

    round 함수 : model list에서 n번째 모델을 하나씩 빼면서 새로운 리스트 만든다

    models = [A, B, C, D]

    models_1 = [B, C, D]
    models_2 = [A, C, D]
    models_3 = [A, B, D]
    models_4 = [A, B, C]

In [ ]:
ensemble = [model_A, model_B, model_C]


ensemble_round = round(ensemble)

# model ensemble

In [ ]:
dic_ae = {
    'id' : [],
    'sentence_form' : [],
    'annotation' : []
}

### 모델 리스트를 입력하세요 ###

models = ensemble

################################

for i in range(len(models[0])):
    tmp_divide = []

    for j in models:  # 각 모델들에 있는 inference를 하나의 리스트에 모은다.
        tmp_divide.append( str(j['annotation'][i]) )

    answer = Counter(tmp_divide).most_common(n=1)[0][0] # 리스트에서 가장 많은 inference를 하드보팅 한다.

    # annotation 유무를 확인한다.
    check =  str(tmp_divide).replace("[", "").replace("]", "").replace(",", "").replace("\'", "").replace("\"", "").replace(" ", "")

    if '[]' in answer and check is not "":   # []이 하드보팅의 결과값이 된다면 다른 inference 값이 있는지 확인하고,
        while '[]' in tmp_divide:            # 다른 inference 값이 있다면, 그 값을 답으로 추론한다.
            tmp_divide.remove('[]')
        dic_ae['annotation'].append( tmp_divide[0] )
    else:
        dic_ae['annotation'].append( answer )
    dic_ae['id'].append( j['id'][i] )
    dic_ae['sentence_form'].append( j['sentence_form'][i] )

df_ae = pd.DataFrame( dic_ae )
df_ae

In [ ]:
# jsonl 파일로 저장하기


### 파일 이름을 입력하세요 ###

file_name = 'ensemble'   

##############################


with open('/content/{0}.jsonl' .format(file_name), 'w') as file:
    for i in range( len(df_ae) ):
        annos = df_ae['annotation'][i]
        str_annos = str(annos)
        tmp = str_annos.replace("None", "null").replace("\'", "\"")

        file.write(  '{'+'\"id\": \"nikluge-sa-2022-{3}-{0}\", \"sentence_form\": \"{1}\", \"annotation\": {2}'\
                    .format( str(i+1).zfill(5)  ,   df_ae['sentence_form'][i], tmp, file_name ) +'}' )
        file.write("\n")

# {"id": "nikluge-sa-2022-dev-00001", "sentence_form": "깔끔하게 부직포 포장으로 되어 있어서 그냥 뜨거운 물에 풍덩 넣어놓고 좀 휘젓어주면 금방 우러난다.", "annotation": [["본품#편의성", ["부직포 포장", 5, 11], "positive"]]}

# 0은 숫자, 1은 sentence, 2는 annotation, 3은 file_name(파일이름)

# model round

In [ ]:
### round 할 모델 리스트를 입력하세요 ###

model_lists = ensemble_round

#########################################

name = 'ensemble_round_'
path = '/content'
# path 마지막에 /는 빼주세요.


for idx, i in enumerate(model_lists):
    model = i
    esb_model = excute_ensemble(model)
    save_jsonl(esb_model, name+str(idx+1), path)

## 잘 열리는지 test

In [ ]:
df_test =  pd.DataFrame(jsonlload("/content/{0}.jsonl". format(file_name)))
df_test

# 또 다른 시도

# 만장일치

만장일치인 정답만 답을 내고, 나머지 annotation은 []

In [ ]:
### 모델 리스트를 입력하세요 ###

models = ensemble

################################


dic_ae = {
    'id' : [],
    'sentence_form' : [],
    'annotation' : []
}


for i in range(len(models[0])):
    tmp_divide = []

    for j in models:        
        tmp_divide.append( str(j['annotation'][i]) )

    tmp_divide = set(tmp_divide)
    
    if len(tmp_divide) == 1:
        dic_ae['annotation'].append( str(j['annotation'][i]) )
    else:
        dic_ae['annotation'].append( '[]' )
    dic_ae['id'].append( j['id'][i] )
    dic_ae['sentence_form'].append( j['sentence_form'][i] )


df_ae = pd.DataFrame( dic_ae )
df_ae

# category, polarity 각각 다수결

In [ ]:
dic_ae = {
    'id' : [],
    'sentence_form' : [],
    'annotation' : []
}

### 모델 리스트를 입력하세요 ###

models = ensemble

################################

for p in range(len(models[0])):
    tmp_list = []
    tmp_divide = []

    ##### cate 다수결
    for j in models:   # tmp_divide = [ ['본품#일반', 'positive'], ['제품 전체#일반', 'positive'], ['본품#일반', 'negative'] ]     
        tmp_divide.append( str(j['annotation'][p]) )   
    dic_ae['id'].append( j['id'][p] )
    dic_ae['sentence_form'].append( j['sentence_form'][p] )   
    str_tmp_divide = str(tmp_divide)

    tmp_cate = re.findall("[가-힣\#\/ ]+", str_tmp_divide)
     # tmp_cate = [ ['본품#일반'], ['제품 전체#일반'], ['본품#일반'] ]    

    for j in range(len(tmp_cate)):
        if ' ' in tmp_cate:
            tmp_cate.remove(' ')
    tmp_cate = list(set(tmp_cate))
     # tmp_cate = [ ['본품#일반'], ['제품 전체#일반'] ]   중복 제거   
    
    picked_cate = []
    for j in tmp_cate:
        cate_count = str_tmp_divide.count(j)
        if cate_count >= len(tmp_divide)//2+1 :   # 과반수 이상이면
            picked_cate.append(j)
            # picked_cate = [ ['본품#일반'] ]   과반수 이상이 가지고 있는 category만 pick

    ##### pola 다수결
    for idx, i in enumerate(picked_cate):
        tmp_answer = []
        pola_pos_count = 0
        pola_neu_count = 0
        pola_neg_count = 0

        for j in tmp_divide:  
            if str(i) in str(j):
                if 'positive' in j:
                    pola_pos_count += 1
                if 'neutral' in j:
                    pola_neu_count += 1
                if 'negative' in j:
                    pola_neg_count += 1
            
        if pola_pos_count >= pola_neu_count:
            if pola_pos_count >= pola_neg_count:
                pola_vote = 'positive'
        elif pola_neu_count > pola_pos_count:
            if  pola_neu_count > pola_neg_count:
                pola_vote = 'neutral'
        else:
            pola_vote = 'negative'

        tmp_answer.append( str(picked_cate[idx]) )
        tmp_answer.append( str(pola_vote) )

        tmp_list.append( tmp_answer )
    
    ##### 하나도 못잡은것 후처리
    check =  str(tmp_divide).replace("[", "").replace("]", "").replace(",", "").replace("\'", "").replace("\"", "").replace(" ", "")
    if '[]' in str(tmp_list) and check is not "":
        while '[]' in tmp_divide:
            tmp_divide.remove('[]')
        dic_ae['annotation'].append( tmp_divide[0] )
    else:
        dic_ae['annotation'].append( tmp_list )

df_ae = pd.DataFrame( dic_ae )
df_ae